In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
len(city_data_df)

566

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# # Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# # Add the heatmap layer.
fig.add_layer(heat_layer)
# # Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Padang,ID,2022-02-04 03:29:55,-0.9492,100.3543,84.25,70,92,2.55
4,4,Vaini,TO,2022-02-04 03:32:29,-21.2000,-175.2000,86.16,79,75,10.36
7,7,Atuona,PF,2022-02-04 03:32:30,-9.8000,-139.0333,78.96,71,3,17.87
9,9,Rikitea,PF,2022-02-04 03:32:31,-23.1203,-134.9692,78.22,74,100,17.02
10,10,Avarua,CK,2022-02-04 03:32:31,-21.2078,-159.7750,82.45,78,75,16.11
...,...,...,...,...,...,...,...,...,...,...
523,523,Road Town,VG,2022-02-04 03:44:53,18.4167,-64.6167,75.99,77,86,18.34
541,541,Porto Velho,BR,2022-02-04 03:46:28,-8.7619,-63.9039,75.25,94,0,1.14
542,542,Tarakan,ID,2022-02-04 03:46:28,3.3000,117.6333,85.50,66,40,8.05
555,555,Kaeo,NZ,2022-02-04 03:47:32,-35.1000,173.7833,78.33,81,96,9.40


In [14]:
preferred_cities_df.count()

City_ID       137
City          137
Country       137
Date          137
Lat           137
Lng           137
Max Temp      137
Humidity      137
Cloudiness    137
Wind Speed    137
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Padang,ID,84.25,-0.9492,100.3543,
4,Vaini,TO,86.16,-21.2000,-175.2000,
7,Atuona,PF,78.96,-9.8000,-139.0333,
9,Rikitea,PF,78.22,-23.1203,-134.9692,
10,Avarua,CK,82.45,-21.2078,-159.7750,
17,Barra Patuca,HN,75.49,15.8000,-84.2833,
18,Kahului,US,79.86,20.8947,-156.4700,
26,Meulaboh,ID,84.00,4.1363,96.1285,
31,Tautira,PF,86.02,-17.7333,-149.1500,
33,Kavieng,PG,83.73,-2.5744,150.7967,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Padang,ID,84.25,-0.9492,100.3543,Mercure Padang
4,Vaini,TO,86.16,-21.2000,-175.2000,Keleti Beach Resort
7,Atuona,PF,78.96,-9.8000,-139.0333,Villa Enata
9,Rikitea,PF,78.22,-23.1203,-134.9692,People ThankYou
10,Avarua,CK,82.45,-21.2078,-159.7750,Paradise Inn
17,Barra Patuca,HN,75.49,15.8000,-84.2833,
18,Kahului,US,79.86,20.8947,-156.4700,Maui Seaside Hotel
26,Meulaboh,ID,84.00,4.1363,96.1285,Kost Pria
31,Tautira,PF,86.02,-17.7333,-149.1500,Pension Oaoa
33,Kavieng,PG,83.73,-2.5744,150.7967,Nusa Island Retreat


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# # Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
# locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
# fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
# marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
# fig.add_layer(marker_layer)

# # Call the figure to plot the data.
# fig

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))